In [1]:
from datetime import datetime 
import numpy as np
import pandas as pd
import math
import cv2
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Dropout,Dense,Activation,Flatten,BatchNormalization,ZeroPadding2D
from keras.preprocessing import image
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
df = pd.read_csv('./input/labels.csv')
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [3]:
breeds = set(df['breed'])
n = len(breeds)
breed_to_number = dict(zip(breeds,range(n)))
number_to_breed = dict(zip(range(n),breeds)) 

In [4]:
n = len(df)
input_shape = (256,256)
X_train = np.zeros((n,256,256,3),dtype = np.float32)
Y_train = np.zeros((n,len(breeds)),dtype = np.int8)
for i in tqdm(range(n)):
    X_train[i] = cv2.resize(cv2.imread('./input/train/%s.jpg'%df['id'][i]),input_shape)
    Y_train[i][breed_to_number[df['breed'][i]]] = 1
X_train = X_train / 255

100%|██████████| 10222/10222 [00:24<00:00, 417.26it/s]


In [5]:
Y_ = np.zeros(n,dtype = np.int8)
for i in tqdm(range(n)):
    Y_[i] = breed_to_number[df['breed'][i]]
Y_ = keras.utils.to_categorical(Y_,len(breeds))
print(Y_)

100%|██████████| 10222/10222 [00:00<00:00, 97324.08it/s]

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [10]:
model = Sequential()
#conv1
model.add(Conv2D(96,kernel_size=(11,11),
                activation='relu',
                strides=4,
                padding = 'valid',
                input_shape=X_train.shape[1:]))
#model.add(BatchNormalization())
#pooling1
model.add(MaxPooling2D(pool_size=(3,3),
                      strides = 2,padding='valid'))
#conv2
model.add(ZeroPadding2D((2,2)))
model.add(Conv2D(256,kernel_size=(5,5),
                activation='relu'))
#model.add(BatchNormalization())
#pooling2
model.add(MaxPooling2D(pool_size=(3,3),
                      strides=2))
#conv3
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(384,kernel_size=(3,3),
                 padding = 'valid',
                 activation='relu'))
#model.add(BatchNormalization())
#conv4
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(384,kernel_size=(3,3),
                 padding='valid',
                 activation='relu'))
#model.add(BatchNormalization())
#conv5
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256,kernel_size=(3,3),
                 padding = 'valid',
                activation='relu'))
#model.add(BatchNormalization())
#pooling3
model.add(MaxPooling2D(pool_size=(3,3),
                      strides=2))
#full connected
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(4096,activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(120))
#model.add(BatchNormalization())

model.add(Activation('softmax'))

log_filepath = '/tmp/keras_log'
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=sgd,
             metrics=['accuracy'])
#tb_cb = keras.callbacks.TensorBoard(log_dir=log_filepath, write_images=1, histogram_freq=1)
#cbks = [tb_cb]

In [11]:
datagen = image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
#datagen.fit(X_train)
h = model.fit(X_train, Y_, batch_size=64, epochs=1000, validation_split=0.05)
#h = model.fit_generator(datagen.flow(X_train,Y_,batch_size = 64),steps_per_epoch = len(X_train) / 64,epochs=1000)

Train on 9710 samples, validate on 512 samples
Epoch 1/1000
9710/9710 [==============================] - 13s 1ms/step - loss: 4.7873 - acc: 0.0104 - val_loss: 4.7838 - val_acc: 0.0078
Epoch 2/1000
9710/9710 [==============================] - 13s 1ms/step - loss: 4.7815 - acc: 0.0114 - val_loss: 4.7874 - val_acc: 0.0020
Epoch 3/1000
9710/9710 [==============================] - 13s 1ms/step - loss: 4.7801 - acc: 0.0124 - val_loss: 4.7847 - val_acc: 0.0020
Epoch 4/1000
9710/9710 [==============================] - 13s 1ms/step - loss: 4.7756 - acc: 0.0107 - val_loss: 4.7826 - val_acc: 0.0078
Epoch 5/1000
9710/9710 [==============================] - 13s 1ms/step - loss: 4.7556 - acc: 0.0141 - val_loss: 4.6949 - val_acc: 0.0176
Epoch 6/1000
9710/9710 [==============================] - 13s 1ms/step - loss: 4.6953 - acc: 0.0208 - val_loss: 4.6287 - val_acc: 0.0254
Epoch 7/1000
9710/9710 [==============================] - 13s 1ms/step - loss: 4.6083 - acc: 0.0280 - val_loss: 4.5370 - val_acc: 0

KeyboardInterrupt: 